In [12]:
import pandas as pd
import yfinance as yf
import sqlite3

In [13]:
conn = sqlite3.connect("../Nexus.db")
df = pd.read_sql_query("SELECT * FROM notifications", conn)
conn.close()

In [14]:
df.head()

,id,company_name,date,market,issuer_id,link,message,pdf_text
0,1,Ayfie International AS.json,04.09.2025 09:05,MERK,AIX,message_654625,Godthåb Holding AS has today bought 105.000 s...,
1,2,Ayfie International AS.json,28.08.2025 10:22,MERK,AIX,message_654077,HAAS AS has today bought 150.000 shares in Ayf...,
2,3,Ayfie International AS.json,09.05.2025 18:17,MERK,AIX,message_645945,Reference is made to the stock exchange announ...,KRT-1500 Skjema for melding om transaksjoner u...
3,4,Ayfie International AS.json,15.01.2025 15:02,MERK,AIX,message_636562,"On 30 December 2024/14th January 2025, a demer...",
4,5,Ayfie International AS.json,30.12.2024 16:12,MERK,AIX,message_635776,HAAS AS has today bought 33.343 shares in Ayfi...,


In [15]:
df["date"] = pd.to_datetime(df["date"], format="%d.%m.%Y %H:%M")
df["date"].min()

Timestamp('2015-01-02 08:10:00')

In [16]:
# As downloding the stock prices is a nettwork task i will continue to use normal python loop.
# This is as to not get error when downloding.
issuer_ids = df["issuer_id"].unique().tolist()

In [17]:
error_list = []
stock_prices = pd.DataFrame()

def get_stock_price(ticker: str) -> pd.DataFrame | None:
    try:
        ticker += ".OL"
        data = yf.download(ticker, period="11y", interval="1d", auto_adjust=True, progress=False)
        
        data.columns = [col[0] for col in data.columns]
        data = data.reset_index()

        data["ticker"] = ticker
        
        return data
    except:
        error_list.append(f"Error with stock {ticker}")

for ticker in issuer_ids:
    stock_price = get_stock_price(ticker)
    
    if stock_price is not None:
        stock_prices = pd.concat([stock_prices, stock_price], ignore_index=True)

HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: SPOBK.OL"}}}

1 Failed download:
['SPOBK.OL']: YFPricesMissingError('possibly delisted; no price data found  (period=11y) (Yahoo error = "No data found, symbol may be delisted")')
HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: NHC.OL"}}}

1 Failed download:
['NHC.OL']: YFPricesMissingError('possibly delisted; no price data found  (period=11y) (Yahoo error = "No data found, symbol may be delisted")')

1 Failed download:
['PLH.OL']: YFPricesMissingError('possibly delisted; no price data found  (period=11y) (Yahoo error = "No data found, symbol may be delisted")')

1 Failed download:
['EISP.OL']: YFPricesMissingError('possibly delisted; no price data found  (period=11y) (Yahoo error = "No data found, symbol may be delisted")')

1 Failed download:
['CSS.OL']: YFPricesMissingError('possibly delisted; no p

In [18]:
stock_prices

,Date,Close,High,Low,Open,Volume,ticker,Adj Close
0,2020-07-07,20.774784,21.174299,17.418858,17.578665,867144.0,AIX.OL,NaN
1,2020-07-08,23.891003,23.891003,21.178295,21.917398,530604.0,AIX.OL,NaN
2,2020-07-09,26.607704,27.966057,24.610130,25.169451,458413.0,AIX.OL,NaN
3,2020-07-10,25.249353,28.769081,23.891003,27.702377,270235.0,AIX.OL,NaN
4,2020-07-13,23.970905,26.367996,22.372845,25.968481,228139.0,AIX.OL,NaN
...,...,...,...,...,...,...,...,...
552158,2025-11-13,258.000000,258.000000,257.950012,257.950012,3461.0,SOGN.OL,NaN
552159,2025-11-14,275.000000,275.000000,262.000000,262.000000,1385.0,SOGN.OL,NaN
552160,2025-11-17,275.000000,282.200012,270.000000,275.000000,222.0,SOGN.OL,NaN
552161,2025-11-18,271.950012,271.950012,270.000000,270.049988,1568.0,SOGN.OL,NaN


In [19]:
conn = sqlite3.connect("../Nexus.db")

stock_prices.to_sql(
    name="stock_prices",
    con=conn,
    if_exists="replace",
    index=False
)

conn.close()

In [20]:
print(len(error_list))
print(error_list)

0
[]
